# TensorFlow crash course
### **PART 5**

### Custom model

In [ ]:
#  Implement a model containing residual blocks
#  Preview of a residual block
#  Dense_1 -> Dense_2 -> Dense_3 -> (Sub_Dense_1 -> Sub_Dense_2 -> Dense_1 + Sub_Dense_2)

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

class ResidualBlock(Layer): #  Residual block implementation
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [Dense(n_neurons, activation="elu",
                        kernel_initializer="he_normal")
                        for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs 
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z


#  We use the subclassing API of keras to create custom model

class ResidualRegressor(Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = Dense(30, activation="elu", 
                             kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)



### Model internal based loss

In [ ]:
#  As we saw in the previous parts we calculate losses and metrics based on y_pred and y_true
#  Sometimes we need to calculate losses and metics using other parts of the model such as weights

#  Reconstruction loss for a regression DNN with 5 layers

class ReconstructingRegressor(Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [Dense(30, activation="selu",
                        kernel_initializer="lecun_normal")
                        for _ in range(5)]
        self.out = Dense(output_dim)

    def build(self, batch_input_shape): #  Build an extra dense layer to reconsturct the inputs
        n_inputs = batch_input_shape[-1]
        self.reconstruct = Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)

